In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qibo import hamiltonians, set_backend
from qibo.derivative import parameter_shift

from qaml_scripts.evolution import generate_adiabatic
from qaml_scripts.training import train_adiabatic_evolution
from qaml_scripts.rotational_circuit import rotational_circuit

set_backend("numpy")

# Determining probability density functions with adiabatic quantum computing [1]

Our goal is to fit PDFs. In a few words, we want a model which, given data $x$ sampled 
from a distribution $\rho$,
returns an estimation of the PDF: $\hat{\rho}(x)$.
For doing this, we start by fitting the Cumulative Density Function $F$ of the sample.

The strategy will be the following:

<img src="figures/pdf_est.png" width="800" height="800" align="center"/>

## 1. CDF and PDF

In [ ]:
# a normal sample
s = np.random.randn(10000)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.title("PDF histogram")
plt.hist(s, bins=20, color="orange", hatch="//", histtype="stepfilled", edgecolor="orange", alpha=0.3, density=True)
plt.hist(s, bins=20, color="orange", alpha=1, lw=1.5, histtype="step", density=True)
plt.xlabel('x')
plt.ylabel(r"$\rho$")

plt.subplot(1,2,2)
plt.title("CDF histogram")
plt.hist(s, bins=20, color="purple", hatch="\\\\", histtype="stepfilled", edgecolor="purple", alpha=0.3, density=True, cumulative=True)
plt.hist(s, bins=20, color="purple", alpha=1, lw=1.5, histtype="step", density=True, cumulative=True)
plt.xlabel('x')
plt.ylabel(r"$F$")

plt.show()

> Fitting the CDF is useful because we can label all the points of the sample and 
> the problem becomes a supervised ML problem. In fact, each point $x_i$ of a sample has an empirical CDF value (the number of points equal smaller than $x_i$), on contrary, by tackling the PDF problem directly, we need to represent the distribution in some way (e.g. with an histogram and a binning), introducing some bias.

Once the CDF is fitted, we can calculate the derivative of the CDF in order to get
the PDF since:

$$ \rho(x) = \frac{\text{d} F(x)}{\text{d} x}. $$

## 2. Adiabatic Evolution in a nutshell

Considering a quantum system set to be in an initial configuration described by $H_0$, 
we call adiabatic evolution (AE) of the system from $H_0$ to $H_1$ ad adiabatic
process governed by the following hamiltonian:
$$ H_{\rm ad}(\tau; \vec{\theta}) = \bigl[1 - s(\tau; \vec{\theta})\bigr] H_0 + 
s(\tau; \vec{\theta}) H_1,$$
where we define $s(\tau; \vec{\theta})$ scheduling function of the evolution. According
to the _adiabatic theorem_, if the process is slow enough, the system remains in
the groundstate of $H_{\rm ad}$ during the evolution.
As scheduling function we are going to use a polynomial function of the form:
$$ s(\tau; \vec{\theta}) = \frac{1}{\Theta} \sum_{k=0}^{N_{\rm params}} \theta_k x^k, 
\qquad \text{with} \qquad \Theta = \sum_{k=1}^{N_{\rm params}} \theta_k,$$
in order to satisfy the computational condition $s(0)=0$ and $s(1)=1$.

## 3. Adiabatic Evolution as tool for encoding CDFs

We are going to use the expected value of a Pauli Z over the evolved state as predictor
of the CDF. This kind of approach can be useful because, when fitting a CDF, we need to 
satisfy some conditions:

1. the CDF is strictly monotonic;
2. $M(x_a) = 0$ and $M(x_b)=1$, with $x_a$ and $x_b$ limits of the random variable 
   $x$ domain.

### Inducing monotonoy

Regarding the first point, as heuristic consideration we can think that an adiabatic 
process is naturally led to follow a "monotonic" behaviour. In particular, if the 
coefficients of the scheduling function are **positive**, we induce the process to 
be monotonic. 

### Exploiting the boundaries 

Secondly, by selecting two hamiltonians $H_0$ and $H_1$ whose energies on the ground state 
satisfy our boundary conditions, we can easily contrain the problem to our desired
requirements.

At this point, we can keep track of some energy (as we said, we use the expected value of a 
Pauli Z) during the evolution $E$. 

> __The goal:__ we perform the mapping: $(x_j, F_j) \to (\tau_j, E_j)$ and we train the
> evolution to let energies pass through our training points ${F_j}$. 

### Adiabatic Evolution settings

We need to define an adiabatic evolution in which encoding a Cumulative Density
Function. We need to set the energy boundaries to $E(0)=0$ and $E(1)=1$.

For doing this, we set $H_0=\hat{X}$ and $H_1=\hat{Z}$. If then we set a Pauli Z to be 
the target observable whose energy is tracked during the evolution:

In [ ]:
hx = hamiltonians.X(nqubits=1)
hz = hamiltonians.Z(nqubits=1)

print(f"Expectation of Z over the ground state of a Z: {hz.expectation(hx.ground_state())}")
print(f"Expectation of Z over the ground state of a Z: {hz.expectation(hz.ground_state())}")

## 4. Set the Adiabatic Evolution 

In [ ]:
# Definition of the Adiabatic evolution

nqubits = 1
finalT = 50
dt = 1e-1

# rank of the polynomial scheduling
nparams = 15

# set hamiltonianas
h0 = hamiltonians.X(nqubits, dense=True)
h1 = hamiltonians.Z(nqubits, dense=True)
# we choose a target observable
obs_target = h1

# ground states of initial and final hamiltonians
gs_h0 = h0.ground_state()
gs_h1 = h1.ground_state()

# energies at the ground states
e0 = obs_target.expectation(gs_h0)
e1 = obs_target.expectation(gs_h1)

print(f"Energy at 0: {e0}")
print(f"Energy at 1: {e1}")

# initial guess for parameters
# picking up from U(0,1) helps to get the monotony
init_params = np.random.uniform(0, 1, nparams)

# Number of steps of the adiabatic evolution
nsteps = int(finalT/dt)

# array of x values, we want it bounded in [0,1]
xarr = np.linspace(0, 1, num=nsteps+1, endpoint=True)
print("\nFirst ten evolution times:")
print(xarr[0:10])

In [ ]:
# generate an adiabatic evolution object and an energy callbacks container
evolution, energy = generate_adiabatic(h0=h0, h1=h1, obs_target=obs_target, dt=dt, params=init_params)
# evolve until final time
_ = evolution(final_time=finalT)

### Some useful plotting functions

In [ ]:
def plot_energy(times, energies, title, cdf=None):
    """Plot energies"""
    plt.figure(figsize=(8,5))
    plt.plot(times, energies, color="purple", lw=2, alpha=0.8, label="Energy callbacks")
    if cdf is not None:
        plt.plot(times, cdf, color="orange", lw=2, alpha=0.8, label="Empirical CDF")
    plt.title(title)
    plt.xlabel(r'$\tau$')
    plt.ylabel("E")
    plt.grid(True)
    plt.legend()
    plt.show()

def show_sample(times, sample, cdf, title):
    """Plot energies"""
    plt.figure(figsize=(8,5))
    plt.hist(sample, bins=50, color="black", alpha=0.3, cumulative=True, density=True, label="Sample")
    plt.plot(times, -cdf, color="orange", lw=2, alpha=0.8, label="Target CDF")
    plt.title(title)
    plt.xlabel(r'$\tau$')
    plt.ylabel("CDF")
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_final_results(times, sample, e, de, title):
    """Plot final results"""
    plt.figure(figsize=(12,4))

    plt.subplot(1,2,1)
    plt.title("PDF histogram")
    plt.hist(sample, bins=20, color="orange", histtype="stepfilled", edgecolor="orange", hatch="//", alpha=0.3, density=True)
    plt.hist(sample, bins=20, color="orange", alpha=1, lw=1.5, histtype="step", density=True)
    plt.plot(times, de, color="purple", lw=2, label=r"Estimated $\rho$")
    plt.xlabel('x')
    plt.ylabel(r"$\rho$")
    plt.legend()

    plt.subplot(1,2,2)
    plt.title("CDF histogram")
    plt.hist(sample, bins=20, color="orange", histtype="stepfilled", edgecolor="orange", hatch="//", alpha=0.3, density=True, cumulative=True)
    plt.hist(sample, bins=20, color="orange", alpha=1, lw=1.5, histtype="step", density=True, cumulative=True)
    plt.plot(times, -np.array(e), color="purple", lw=2, label=r"Estimated $F$")
    plt.xlabel('x')
    plt.ylabel(r"$F$")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_energy(xarr, energy.results, "Not trained evolution")

## 5. Sample from a distribution

We sample a target dataset from a Gamma distribution.

In [ ]:
def cdf_fun(xarr):
    """Generate a sample of data following a Gamma distribution."""
    
    nvals = 10000
    shape = 10
    scale = 0.5
    
    sample = np.random.gamma(shape, scale, nvals)

    normed_sample = (sample - np.min(sample)) / (np.max(sample) - np.min(sample)) 

    h, b = np.histogram(normed_sample, bins=nsteps, range=[0,1], density=False)
    # Sanity check
    np.testing.assert_allclose(b, xarr)

    cdf_raw = np.insert(np.cumsum(h)/len(h), 0, 0)

    # Translate the CDF such that it goes from 0 to 1
    cdf_norm = (cdf_raw - np.min(cdf_raw)) / (np.max(cdf_raw) - np.min(cdf_raw))
    # And now make it go from the E_initial to E_final (E0 to E1)
    cdf = e0 + cdf_norm*(e1 - e0)

    return cdf, sample, normed_sample

In [ ]:
cdf, sample, normed_sample = cdf_fun(xarr)

### A look to the CDF

In [ ]:
show_sample(times=xarr, sample=normed_sample, cdf=cdf, title="Target Cumulative Density Function")
plot_energy(times=xarr, energies=energy.results, title="Callbacks VS eCDF", cdf=cdf)

## 6. Train the evolution to follow the CDF

The training procedure is the following:

1. we fill the scheduling with a set of parameters;
2. we perform the evolution with the defined set of parameters and we collect all the energies $\{E_k\}_{k=1}^{N_{\rm data}}$, where $E_k$ is the expected value of Z over the evolved state at $\tau_k \equiv x_k$.
3. we calculate a loss function:
   $$ J_{\rm mse} = \frac{1}{N_{\rm data}} \sum_{k=1}^{N_{\rm data}} \bigl[E_k - F(x_k)\bigr]^2. $$
4. we use the CMA-ES optimizer to find the best set of parameters of the scheduling which lead the energy of Z to pass through
   the CDF values.

In [ ]:
best_params = train_adiabatic_evolution(
    nsteps=nsteps,
    xarr=xarr,
    cdf=cdf,
    training_n=20,
    init_params=init_params,
    e0=e0,
    e1=e1,
    target_loss=1e-3,
    finalT=finalT,
    h0=h0,
    h1=h1,
    obs_target=obs_target
)

## 7. From Adiabatic Evolution to a quantum circuit

We define an unitary operator which can be used to get the evolved state at any 
evolution time $\tau$ thanks to some calculations on the evolution operator
associated to $H_{\rm ad}$. This operator is translated into a circuit composed 
of some rotations in `rotational_circuit.py`.

> If we are able to translate the problem to a VQC composed of rotations, we can
> use the Parameter Shift Rule to derivate it very easily.

In [ ]:
rotcirc = rotational_circuit(best_p=best_params, finalT=finalT)

In [ ]:
# a look to the circuit
# this circuit must be filled with a time value to be well defined

circ1 = rotcirc.rotations_circuit(t=0.1)
circ2 = rotcirc.rotations_circuit(t=0.8)

print(f"Circuit diagram: {circ1.draw()}")
print(f"\nCirc1 params: {circ1.get_parameters()}")
print(f"\nCirc2 params: {circ2.get_parameters()}")

### Calculate derivative of the circuit with respect to $\tau$

In [ ]:
def psr_energy(t, nshots=None):
    """Calculate derivative of the energy with respect to the real time t."""

    c = rotcirc.rotations_circuit(t)
    dd1, dd2, dd3 = rotcirc.derivative_rotation_angles(t)

    d1 = dd1 * parameter_shift(circuit=c, parameter_index=0, hamiltonian=obs_target, nshots=nshots)
    # looking inside the circuit you will see the second angle is filled with a "-" before
    d2 = - dd2 * parameter_shift(circuit=c, parameter_index=1, hamiltonian=obs_target, nshots=nshots)
    d3 = dd3 * parameter_shift(circuit=c, parameter_index=2, hamiltonian=obs_target, nshots=nshots)

    return (d1 + d2 + d3)

### Calculate derivatives time by time

In [ ]:
nshots = 1000
real_times = np.linspace(0, finalT, len(xarr))

de = []
e = []

# loop over times
for t in real_times:
    c = rotcirc.rotations_circuit(t)
    exp = obs_target.expectation(c.execute(nshots=nshots).state())
    # to avoid numerical instabilities when close to zero
    if exp > 0:
        e.append(-exp)
    else:
        e.append(exp)
    de.append(psr_energy(t))

de = - np.asarray(de)*finalT

In [ ]:
plot_final_results(times=xarr, sample=normed_sample, e=e, de=de, title="Final estimations")

#### References

[1] [_Determining probability density functions with adiabatic quantum computing_, 2023](https://arxiv.org/abs/2303.11346)